<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/master/armado_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 2

## Armado del dataset

In [13]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

--2020-06-29 23:00:50--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.200.128, 2607:f8b0:4001:c16::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.200.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz.2’

speech_commands_v0. 100%[===================>]   1.39G   165MB/s    in 8.4s    

2020-06-29 23:00:59 (169 MB/s) - ‘speech_commands_v0.01.tar.gz.2’ saved [1489096277/1489096277]

mkdir: cannot create directory ‘speechcommands’: File exists


In [14]:
# Import de Librerías
import librosa
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

In [15]:
# Busco y guardo en un mapa todos los audio files correspondientes a los digit_keys
digit_command_keys = [
  'zero',                    
  'one',
  'two',
  'three',
  'four',
  'five',
  'six',
  'seven',
  'eight',
  'nine'
]

audio_files = {}

for key in digit_command_keys:
  directory = f'speechcommands/{key}/*.wav'
  audio_files[key] = glob.glob(directory)

In [16]:
# Utilizamos la función propuesta en el TP 
# para hacer la extracción de los datos 
# correspondientes a cada audio file

def calculate_features(
    filename,
    n_mfcc=12,
    delta=True,
    deltadelta=True,
    energy=True, 
    summary_fn = [np.mean, np.std], 
    summary_names=['mean','std']):
  #Abro el archivo:
  x, sr = librosa.core.load(filename,sr=None)
  
  #Calculo MFCCs
  features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
  feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
  #Calculo energia:
  if energy:
    energy = librosa.feature.rmse(x)
    features = np.concatenate([features,energy])
    feat_names = feat_names + ['energy']
  #Aplico media y desvio estandar por defecto
  summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
  feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]
  
  #Lo mismo con los delta
  if delta:
    deltafeatures = np.diff(features)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
    d_names = ['d{}'.format(name) for name in feat_names]
  else:
    d_names = []

  #Y con los delta de segundo orden
  if deltadelta:
    deltadeltafeatures = np.diff(features,n=2)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
    dd_names = ['dd{}'.format(name) for name in feat_names]
  else:
    dd_names = []

  feat_names = feat_names + d_names + dd_names

  return summary_features, feat_names  

In [17]:
audio_df = []

for key in audio_files:
  for audio in audio_files[key]:
    feat, names = calculate_features(audio)
    audio_row = dict(zip(names, feat))
    audio_row['digit'] = key

    audio_df.append(audio_row)
  
audio_df = pd.DataFrame(audio_df)

In [18]:
audio_df

,mfcc_0_mean,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,mfcc_11_mean,energy_mean,mfcc_0_std,mfcc_1_std,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,mfcc_10_std,mfcc_11_std,energy_std,dmfcc_0_mean,dmfcc_1_mean,dmfcc_2_mean,dmfcc_3_mean,dmfcc_4_mean,dmfcc_5_mean,dmfcc_6_mean,dmfcc_7_mean,dmfcc_8_mean,dmfcc_9_mean,dmfcc_10_mean,dmfcc_11_mean,denergy_mean,dmfcc_0_std,dmfcc_1_std,dmfcc_2_std,dmfcc_3_std,dmfcc_4_std,dmfcc_5_std,dmfcc_6_std,dmfcc_7_std,dmfcc_8_std,dmfcc_9_std,dmfcc_10_std,dmfcc_11_std,denergy_std,ddmfcc_0_mean,ddmfcc_1_mean,ddmfcc_2_mean,ddmfcc_3_mean,ddmfcc_4_mean,ddmfcc_5_mean,ddmfcc_6_mean,ddmfcc_7_mean,ddmfcc_8_mean,ddmfcc_9_mean,ddmfcc_10_mean,ddmfcc_11_mean,ddenergy_mean,ddmfcc_0_std,ddmfcc_1_std,ddmfcc_2_std,ddmfcc_3_std,ddmfcc_4_std,ddmfcc_5_std,ddmfcc_6_std,ddmfcc_7_std,ddmfcc_8_std,ddmfcc_9_std,ddmfcc_10_std,ddmfcc_11_std,ddenergy_std,digit
0,-361.668962,53.859608,-6.401287,10.504276,-3.063851,6.977546,-10.760357,9.866350,-7.370083,5.051000,-8.156331,6.235290,0.020506,84.485822,36.688378,32.587302,27.525683,12.506436,18.707495,18.116925,8.329306,12.290994,7.297515,6.983674,5.416876,0.021759,0.359901,0.330505,0.226528,-0.021894,-0.114450,-0.000125,0.052082,0.350795,0.457632,0.267502,0.047328,0.106565,1.676937e-05,23.321771,14.697988,14.023375,13.346168,7.113900,8.144410,7.238074,6.321893,8.587937,5.851012,5.810887,4.690040,0.005552,-0.244996,-0.182200,0.016071,0.149166,0.174072,-0.114410,-0.093302,0.108464,0.012912,-0.257687,-0.302846,-0.123361,-3.074362e-06,16.897698,15.136910,10.813695,11.576804,7.798867,8.156207,5.955693,6.824333,9.274772,7.424050,7.378358,6.327442,0.002906,zero
1,-344.429501,62.971646,2.681298,32.594575,8.551349,3.027861,-19.559357,-7.843833,-16.635529,2.141704,-11.552719,-4.756587,0.118753,173.649231,95.038258,53.529298,42.021534,24.892815,35.768404,22.211827,24.265273,15.124858,12.671261,13.990900,9.912509,0.123166,0.260390,0.334700,0.246556,0.131604,0.014532,-0.091141,-0.176278,-0.228289,-0.235325,-0.197977,-0.132908,-0.062320,-2.054474e-05,56.698277,33.695692,24.187970,20.884370,10.510898,15.940563,11.625681,12.151329,11.425712,8.151023,6.990844,8.121423,0.035362,-0.910990,0.726239,-0.735316,-0.120967,0.361948,-0.793723,0.870366,-0.597435,0.426952,0.141512,-0.224778,0.492211,-6.767740e-04,44.595048,34.310766,19.642012,22.839959,9.655708,13.889084,14.058253,10.832590,14.331551,11.756045,7.560719,11.408998,0.020434,zero
2,-381.906240,131.083215,2.012445,-2.563179,-15.922072,-7.889542,-20.937417,-0.064021,-10.726910,-9.290048,-16.011377,0.984315,0.042835,149.161164,76.830646,36.360022,31.036041,20.566772,18.434693,20.981280,12.833288,9.960668,8.793140,15.277377,6.334916,0.038487,-5.321107,2.195576,-1.007546,-0.641226,1.343937,-0.987439,0.099842,1.208208,0.609522,0.137784,0.701948,-0.386948,-2.833134e-04,31.632638,23.050736,15.745608,10.699331,9.154738,8.849845,9.485480,7.940487,5.662432,5.191579,5.311871,4.591579,0.010598,-2.742922,-0.456658,-0.751993,0.126027,0.180305,0.301520,0.225588,-0.150963,0.048313,0.099245,0.260038,0.153182,-4.064343e-05,16.727111,17.354693,14.925466,9.995126,8.587932,7.835087,9.674393,8.459633,6.429986,6.019982,6.302291,6.111948,0.006472,zero
3,-329.006034,89.473208,-8.629773,41.554231,9.980941,-7.126969,-1.424785,-17.449013,9.463047,4.690114,-24.510781,-4.587081,0.094111,184.086326,85.194146,42.588011,36.838445,20.480218,21.511364,16.249922,17.228201,14.757756,7.775876,14.650661,11.611371,0.101597,-0.902751,0.393178,1.436376,-0.065834,-0.407902,0.427270,0.805424,0.465727,-0.252292,0.078161,0.532303,-0.077415,-4.421020e-06,57.933840,31.254174,18.373933,19.715451,13.388871,10.480729,8.742591,10.220457,7.332688,6.213142,7.831121,7.793500,0.025616,-1.316537,-0.830679,-0.070027,-0.317255,0.099754,0.125140,-0.098501,0.102492,-0.181358,-0.120128,0.404970,0.299215,-3.218973e-05,42.029108,28.110167,17.748159,23.588933,13.925842,11.329843,9.734676,12.

In [19]:
# Guardamos nuestro dataframe en un csv
audio_df.to_csv(r'/content/audio_digits.csv')